In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt
import sys

In [2]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR

In [3]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline
from torch.utils.tensorboard import SummaryWriter

In [4]:
import subprocess
import time
import os
from copy import deepcopy
import math as m
import gc

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [7]:
flow = torch.load("best_flow_model.pt", map_location=device)
flow.eval()

Flow(
  (_transform): CompositeTransform(
    (_transforms): ModuleList(
      (0): RandomPermutation()
      (1): MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        (autoregressive_net): MADE(
          (initial_layer): MaskedLinear(in_features=8, out_features=250, bias=True)
          (blocks): ModuleList(
            (0): MaskedFeedforwardBlock(
              (linear): MaskedLinear(in_features=250, out_features=250, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (1): MaskedFeedforwardBlock(
              (linear): MaskedLinear(in_features=250, out_features=250, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (final_layer): MaskedLinear(in_features=250, out_features=376, bias=True)
        )
      )
      (2): RandomPermutation()
      (3): MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        (autoregressive_net): MADE(
          (initial_layer): MaskedLinear(i

In [8]:
n_samples = int(1e6)
n_sample_size = int(1e4)

steps = int(n_samples/n_sample_size)

generated = None
with torch.no_grad():
    i = 0
    while generated is None or generated.shape[0] < n_samples: 
        try:
            new_data = flow.sample(n_sample_size).cpu().numpy()
        except AssertionError:
            continue
        i += 1
        print("Sampled {} datapoints ({} / {})...".format(n_sample_size, i, steps))
        if generated is None:
            generated = new_data
        else:
            generated = np.vstack((generated, new_data))

Sampled 10000 datapoints (1 / 100)...
Sampled 10000 datapoints (2 / 100)...
Sampled 10000 datapoints (3 / 100)...
Sampled 10000 datapoints (4 / 100)...
Sampled 10000 datapoints (5 / 100)...
Sampled 10000 datapoints (6 / 100)...
Sampled 10000 datapoints (7 / 100)...
Sampled 10000 datapoints (8 / 100)...
Sampled 10000 datapoints (9 / 100)...
Sampled 10000 datapoints (10 / 100)...
Sampled 10000 datapoints (11 / 100)...
Sampled 10000 datapoints (12 / 100)...
Sampled 10000 datapoints (13 / 100)...
Sampled 10000 datapoints (14 / 100)...
Sampled 10000 datapoints (15 / 100)...
Sampled 10000 datapoints (16 / 100)...
Sampled 10000 datapoints (17 / 100)...
Sampled 10000 datapoints (18 / 100)...
Sampled 10000 datapoints (19 / 100)...
Sampled 10000 datapoints (20 / 100)...
Sampled 10000 datapoints (21 / 100)...
Sampled 10000 datapoints (22 / 100)...
Sampled 10000 datapoints (23 / 100)...
Sampled 10000 datapoints (24 / 100)...
Sampled 10000 datapoints (25 / 100)...
Sampled 10000 datapoints (26 / 100

In [9]:
np.savetxt('data/generated_samples.csv', generated, delimiter=',')

In [10]:
generated

array([[0.36135912, 0.5483361 , 0.46152318, ..., 0.6337303 , 0.24384691,
        0.1820904 ],
       [0.49947628, 0.63304967, 0.859351  , ..., 0.56092787, 0.8015759 ,
        0.42879358],
       [0.47925913, 0.60327053, 0.47741085, ..., 0.6178133 , 0.8045587 ,
        0.09474954],
       ...,
       [0.48993105, 0.7020711 , 0.8633891 , ..., 0.698539  , 0.73050636,
        0.00476339],
       [0.49712992, 0.6702369 , 0.21515128, ..., 0.6658591 , 0.17378782,
        0.30985427],
       [0.513977  , 0.4816181 , 0.9559124 , ..., 0.5112317 , 0.97640383,
        0.56917185]], dtype=float32)